# Covid-19 Wastewater Analysis

SARS-CoV-2 has been a major public health crises for almost three years. (expand)


Wastewater data is collected..... (expand on how it's collected)

This data is valuable because it is anonymous and does not rely on individual reporting. It does not require a person to know they have the virus. Many people do not confirm their cases with a government reported test because they do not realize they have Covid19, or they take a rapid home test and do not use reporting tools on government websites, so their cases are not offically counted. This makes public health measures difficult for stakeholders to navigate. Wastewater data is valuable because  health officials can use this data to make decisions and request funding as well as plan for spikes in cases so that they can ready medical facilities for impending increases. 

The limitations of this data are... (expand on limitations and areas that don't collect, and how different cites have started collected at different times, some so recent they did not capture the January 2022 spike, which was the highest so far)


In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import seaborn as sns
import datetime

Our data is available on the Centers for Disease Control and Prevention (CDC) website here https://data.cdc.gov/Public-Health-Surveillance/NWSS-Public-SARS-CoV-2-Wastewater-Metric-Data/2ew6-ywp6 

In [135]:
df = pd.read_csv("NWSS_Public_SARS-CoV-2_Wastewater_Metric_Data (3).csv", lineterminator='\n', on_bad_lines="warn", low_memory=False)

In [136]:
df.head(n=100)

,wwtp_jurisdiction,wwtp_id,reporting_jurisdiction,sample_location,sample_location_specify,key_plot_id,county_names,county_fips,population_served,date_start,date_end,ptc_15d,detect_prop_15d,percentile,sampling_prior,first_sample_date
0,Arkansas,1548.0,Arkansas,Treatment plant,NaN,CDC_BIOBOT_ar_1548_Treatment plant_raw wastewater,Jefferson,05069,42323,2022-09-11,2022-09-25,NaN,100.0,100.0,no,2022-09-25
1,Arkansas,1548.0,Arkansas,Treatment plant,NaN,CDC_BIOBOT_ar_1548_Treatment plant_raw wastewater,Jefferson,05069,42323,2022-09-12,2022-09-26,NaN,100.0,100.0,no,2022-09-25
2,Arkansas,1548.0,Arkansas,Treatment plant,NaN,CDC_BIOBOT_ar_1548_Treatment plant_raw wastewater,Jefferson,05069,42323,2022-09-13,2022-09-27,-98.0,100.0,91.5,no,2022-09-25
3,Arkansas,1548.0,Arkansas,Treatment plant,NaN,CDC_BIOBOT_ar_1548_Treatment plant_raw wastewater,Jefferson,05069,42323,2022-09-14,2022-09-28,-98.0,100.0,91.5,no,2022-09-25
4,Arkansas,1548.0,Arkansas,Treatment plant,NaN,CDC_BIOBOT_ar_1548_Treatment plant_raw wastewater,Jefferson,05069,42323,2022-09-15,2022-09-29,-98.0,100.0,91.5,no,2022-09-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Indiana,1162.0,Indiana,Treatment plant,NaN,CDC_BIOBOT_in_1162_Treatment plant_raw wastewater,Hamilton,18057,98000,2022-07-01,2022-07-15,92.0,100.0,83.6,no,2022-06-01
96,Indiana,1162.0,Indiana,Treatment plant,NaN,CDC_BIOBOT_in_1162_Treatment plant_raw wastewater,Hamilton,18057,98000,2022-07-02,2022-07-16,92.0,100.0,83.6,no,2022-06-01
97,Indiana,1162.0,Indiana,Treatment plant,NaN,CDC_BIOBOT_in_1162_Treatment plant_raw wastewater,Hamilton,18057,98000,2022-07-03,2022-07-17,92.0,100.0,83.6,no,2022-06-01
98,Indiana,1162.0,Indiana,Treatment plant,NaN,CDC_BIOBOT_in_1162_Treatment plant_raw wastewater,Hamilton,18057,98000,2022-07-04,2022-07-18,8.0,100.0,77.2,no,2022-06-01


Variable Descriptions: 
wwtp_jurisdiction: US State the wastewater treatment plant in 'wwtp_id' is located	
wwtp_id: Wastewater treatment plants
reporting_jurisdiction:	The CDC ELC jurisdiction, a state, reporting the data
sample_location:Sample collection location in the wastewater system	
sample_location_specify: "Upstream" sample locations	
key_plot_id: geographic area served by sampling sites (sewersheds)	
county_names: County and county-equivalent names corresponding to the FIPS codes	
county_fips: 5-digit numberic FIP codes all counties and county-equivalents served by the sampling site	
population_served: Estimated number of persons served by the sampling site	
date_start:	The start date of the interval over which metric is calculated
date_end: The end date of the interval over which metric is calculated
ptc_15d: The percent change in SARS-CoV-2 RNA levels over the 15-day interval defined by 'date_start' and 'date_end'
detect_prop_15d: The proportion of tests with SARS-CoV-2 detected, meaning a cycle threshold (Ct) value <40 for RT-qPCR or at least 3 positive droplets/partitions for RT-ddPCR, by sewershed over the 15-day window defined by 'date_start' and "date_end'
percentile:	This metric shows whether SARS-CoV-2 virus levels at a site are currently higher or lower than past historical levels at the same site. 
sampling_prior:	Indicates whether the site was collecting wastewater samples before or on January 13, 2022.
first_sample_date: The first date samples were collected at a site.
 


In [137]:
print("The shape of the data is :",df.shape)

The shape of the data is : (361669, 16)


Changed data type of columns wwtp_id and sample_location_specify from float64 to object. 

In [138]:
change_datatype_wwtp_id = {'wwtp_id': str}
df = df.astype(change_datatype_wwtp_id)

change_datatype_sample_location_specify = {'sample_location_specify' : str}
df = df.astype(change_datatype_sample_location_specify) 

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361669 entries, 0 to 361668
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   wwtp_jurisdiction        361669 non-null  object 
 1   wwtp_id                  361669 non-null  object 
 2   reporting_jurisdiction   361669 non-null  object 
 3   sample_location          361669 non-null  object 
 4   sample_location_specify  361669 non-null  object 
 5   key_plot_id              361669 non-null  object 
 6   county_names             361669 non-null  object 
 7   county_fips              361669 non-null  object 
 8   population_served        361669 non-null  int64  
 9   date_start               361669 non-null  object 
 10  date_end                 361669 non-null  object 
 11  ptc_15d                  324727 non-null  float64
 12  detect_prop_15d          344881 non-null  float64
 13  percentile               263534 non-null  float64
 14  samp

In [142]:
df.describe()

,population_served,ptc_15d,detect_prop_15d,percentile
count,3.616690e+05,3.247270e+05,344881.000000,263534.000000
mean,1.200851e+05,3.491184e+06,92.062419,68.950996
std,2.996939e+05,8.230352e+07,21.954981,134.156220
min,9.000000e+02,-1.000000e+02,0.000000,0.000000
25%,1.120000e+04,-5.700000e+01,100.000000,32.000000
50%,3.071400e+04,0.000000e+00,100.000000,51.667000
75%,9.285000e+04,1.290000e+02,100.000000,70.000000
max,4.000000e+06,2.147484e+09,100.000000,999.000000


Percentile Column has a max of 999.00, which should not be possible. We will check how many values in that column are over 100, which should be the maximum percentile logically. 

In [178]:
percentile_outliers = df[df.percentile > 100]
percentile_outliers.describe()

,population_served,ptc_15d,detect_prop_15d,percentile
count,5208.000000,1.484000e+03,2431.000000,5208.0
mean,57320.741935,4.064048e+07,92.370218,999.0
std,87840.618859,2.922678e+08,23.757856,0.0
min,3000.000000,-1.000000e+02,0.000000,999.0
25%,6108.000000,-7.200000e+01,100.000000,999.0
50%,25000.000000,-6.000000e+00,100.000000,999.0
75%,55000.000000,2.520000e+02,100.000000,999.0
max,564563.000000,2.147484e+09,100.000000,999.0


We need to drop the percentile column outliers so that they do not skew the data. 

In [183]:
df.drop(df[df['percentile'] > 100].index, inplace = True)
df.describe()

,population_served,ptc_15d,detect_prop_15d,percentile
count,3.564610e+05,3.232430e+05,342450.000000,258326.000000
mean,1.210021e+05,3.320633e+06,92.060234,50.200675
std,3.015917e+05,8.004189e+07,21.941675,23.882641
min,9.000000e+02,-1.000000e+02,0.000000,0.000000
25%,1.130000e+04,-5.700000e+01,100.000000,31.667000
50%,3.071400e+04,0.000000e+00,100.000000,51.000000
75%,9.285000e+04,1.280000e+02,100.000000,69.000000
max,4.000000e+06,2.147484e+09,100.000000,100.000000


In [184]:
df.isnull().sum()

wwtp_jurisdiction              0
wwtp_id                        0
reporting_jurisdiction         0
sample_location                0
sample_location_specify        0
key_plot_id                    0
county_names                   0
county_fips                    0
population_served              0
date_start                     0
date_end                       0
ptc_15d                    33218
detect_prop_15d            14011
percentile                 98135
sampling_prior                 0
first_sample_date              0
dtype: int64

Convert columns with dates from string object to date data type, which will let us sort and filter by date:

In [185]:
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])
df['first_sample_date'] = pd.to_datetime(df['first_sample_date'])

Make sure that dates are no longer strings:

In [186]:
df.info()
# We can see they now have the Dtype 'datetime64'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356461 entries, 0 to 361668
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   wwtp_jurisdiction        356461 non-null  object        
 1   wwtp_id                  356461 non-null  object        
 2   reporting_jurisdiction   356461 non-null  object        
 3   sample_location          356461 non-null  object        
 4   sample_location_specify  356461 non-null  object        
 5   key_plot_id              356461 non-null  object        
 6   county_names             356461 non-null  object        
 7   county_fips              356461 non-null  object        
 8   population_served        356461 non-null  int64         
 9   date_start               356461 non-null  datetime64[ns]
 10  date_end                 356461 non-null  datetime64[ns]
 11  ptc_15d                  323243 non-null  float64       
 12  detect_prop_15d 

Covid19 cases reached a historical high, or "spike", January 16, 2022. Many wastewater testing sides were not collecting yet at that time. We will now filter out rows that have a "first_sample_date" value that is after January 16, 2022. 

Box Plot **here**

In [197]:
spike = pd.to_datetime('1/16/2022')

In [198]:
recorded_spike = df.loc[df.first_sample_date <= spike, :] #This data frame will only include rows with collection sites that 
#were recording before the spike

did_not_record_spike = df.loc[df.first_sample_date > spike, :] #This data frame will only include rows with collection sites that do not 
#have the spike on record because they started recording afterward.

Now we check to make sure that the filters worked correctly:

In [199]:
recorded_spike.sort_values(by='first_sample_date', ascending = False).head()
#We see here that by sorting by "first_sample_date" and showing the most recent values that they have the value of 1/13/22

,wwtp_jurisdiction,wwtp_id,reporting_jurisdiction,sample_location,sample_location_specify,key_plot_id,county_names,county_fips,population_served,date_start,date_end,ptc_15d,detect_prop_15d,percentile,sampling_prior,first_sample_date
335769,Ohio,74.0,Ohio,Treatment plant,nan,NWSS_oh_74_Treatment plant_raw wastewater,Crawford,39033,12362,2022-04-03,2022-04-17,167.0,0.0,10.8,no,2022-01-16
108041,Ohio,157.0,Ohio,Treatment plant,nan,NWSS_oh_157_Treatment plant_raw wastewater,Sandusky,39143,25000,2022-07-03,2022-07-17,773.0,100.0,74.5,no,2022-01-16
335871,Ohio,74.0,Ohio,Treatment plant,nan,NWSS_oh_74_Treatment plant_raw wastewater,Crawford,39033,12362,2022-07-14,2022-07-28,-37.0,100.0,75.0,no,2022-01-16
335870,Ohio,74.0,Ohio,Treatment plant,nan,NWSS_oh_74_Treatment plant_raw wastewater,Crawford,39033,12362,2022-07-13,2022-07-27,-37.0,100.0,75.0,no,2022-01-16
335869,Ohio,74.0,Ohio,Treatment plant,nan,NWSS_oh_74_Treatment plant_raw wastewater,Crawford,39033,12362,2022-07-12,2022-07-26,-37.0,100.0,75.0,no,2022-01-16


In [200]:
did_not_record_spike.sort_values(by='first_sample_date').head()
#We see here that by sorting by "first_sample_date" and showing the oldest values that the earliest "first_sample_date"
#in this group is 1/16/22, which was after the spike. This means our filters worked correctly, and our variables hold 
#the correct data. 

,wwtp_jurisdiction,wwtp_id,reporting_jurisdiction,sample_location,sample_location_specify,key_plot_id,county_names,county_fips,population_served,date_start,date_end,ptc_15d,detect_prop_15d,percentile,sampling_prior,first_sample_date
262466,Wisconsin,313.0,Wisconsin,Treatment plant,nan,NWSS_wi_313_Treatment plant_raw wastewater,Marinette,55075,3500,2022-10-14,2022-10-28,7.0,100.0,73.667,no,2022-01-17
4228,Ohio,315.0,Ohio,Treatment plant,nan,NWSS_oh_315_Treatment plant_raw wastewater,Miami,39109,20581,2022-01-12,2022-01-26,-78.0,100.0,92.000,no,2022-01-17
4229,Ohio,315.0,Ohio,Treatment plant,nan,NWSS_oh_315_Treatment plant_raw wastewater,Miami,39109,20581,2022-01-13,2022-01-27,-78.0,100.0,92.000,no,2022-01-17
4230,Ohio,315.0,Ohio,Treatment plant,nan,NWSS_oh_315_Treatment plant_raw wastewater,Miami,39109,20581,2022-01-14,2022-01-28,-78.0,100.0,92.000,no,2022-01-17
4231,Ohio,315.0,Ohio,Treatment plant,nan,NWSS_oh_315_Treatment plant_raw wastewater,Miami,39109,20581,2022-01-15,2022-01-29,-78.0,100.0,92.000,no,2022-01-17


In [201]:
did_not_record_spike.describe()

,population_served,ptc_15d,detect_prop_15d,percentile
count,1.428020e+05,1.288820e+05,136564.000000,133955.000000
mean,1.255719e+05,5.368741e+06,92.341723,49.919109
std,2.966138e+05,1.026523e+08,21.442050,22.539145
min,9.000000e+02,-1.000000e+02,0.000000,0.000000
25%,1.124600e+04,-5.100000e+01,100.000000,32.500000
50%,3.047900e+04,0.000000e+00,100.000000,50.800000
75%,9.047400e+04,1.090000e+02,100.000000,67.667000
max,4.000000e+06,2.147484e+09,100.000000,100.000000


In [202]:
#scatterplot
percentile_did_not_record_spike = did_not_record_spike.percentile.quantile(.75)
print(percentile_did_not_record_spike)

67.667


In [203]:
recorded_spike.describe()

,population_served,ptc_15d,detect_prop_15d,percentile
count,2.136590e+05,1.943610e+05,205886.000000,124371.000000
mean,1.179479e+05,1.962520e+06,91.873522,50.503940
std,3.048360e+05,6.052247e+07,22.264982,25.246412
min,2.273000e+03,-1.000000e+02,0.000000,0.000000
25%,1.130000e+04,-6.000000e+01,100.000000,30.457000
50%,3.225500e+04,-1.000000e+00,100.000000,51.200000
75%,9.500000e+04,1.430000e+02,100.000000,70.500000
max,4.000000e+06,2.147484e+09,100.000000,100.000000


In [204]:
#scatterplot
percentile_recorded_spike = recorded_spike.percentile.quantile(.75)
print(percentile_recorded_spike)

70.5


What percent of these sites have been collecting samples since before the spike? First we have to find out how many unique testing locations are in each group- those that recorded the spike and those that didnt:

In [205]:
print('Unique testing locations that have been recording since before the spike: ',len(recorded_spike['wwtp_id'].unique().tolist()))
print()
print('Unique testing locations that have been recording since before the spike: ',len(did_not_record_spike['wwtp_id'].unique().tolist()))

Unique testing locations that have been recording since before the spike:  370

Unique testing locations that have been recording since before the spike:  722


As we can see, we have 370 testing sites that recorded the 

Pie chart **here** w/labels that represent those numbers 

Now we will look at the most recent test results, starting with October 1, 2022.

In [206]:
recent = pd.to_datetime('10/01/2022') #defining "recent" as October 1st, 2022

We will create a data frame for recent tests from the sites that did record the spike:

In [94]:
recent_test_recorded_spike = recorded_spike.loc[recorded_spike.date_start >= recent, :] 

Now we will create a data frame for recent tests from the sites that did not record the spike:

In [95]:
recent_test_did_not_record_spike = did_not_record_spike.loc[did_not_record_spike.date_start >= recent, :]

Question: Did not recording the very high case counts during the spike affect the percentile values for testing sites that did not record high case counts previously? 

In [104]:
print('Recent percentile mean for sites that did record the spike: ',recent_test_recorded_spike.percentile.mean())
print()
print('We can see that for testing sites that recorded very high values previously that current levels are still quite \
low in comparison')

Recent percentile mean for sites that did record the spike:  41.749500294348444

We can see that for testing sites that recorded very high values previously that current levels are still quite low in comparison


In [107]:
print('Recent percentile mean for sites that did not record the spike: ',recent_test_did_not_record_spike.percentile.mean())
print()
print('We can see that for testing sites that did not record very high values previously that current levels \
are quite high in comparison')

Recent percentile mean for sites that did not record the spike:  118.93582263735414

We can see that for testing sites that did not record very high values previously that current levels are quite high in comparison
